# 回归中的拟合问题
![回归拟合](image/class5_1.png)

# 分类中的拟合问题
![分类中的拟合](image/class5_2.png)

# 解决过拟合的方式
![解决过拟合的方式](image/class5_3.png)
- dropout就是在训练的过程中,每次迭代的时候随机使得有些神经元工作,有些神经元不工作
- 测试的时候会使用其所有的神经元进行测试

In [2]:
# dropout解决过拟合的实例
# 手写数字识别案例(线性分类器)
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
# 载入数据集。第一个参数是路径名，第二个是采用的编码
mnist = input_data.read_data_sets("/Users/yuejinxiong/MNIST_data",one_hot=True)

# 每个批次的大小(每次放入100张图片去训练)
batch_size = 100
# 计算一共有多少个批次
n_batch = mnist.train.num_examples // batch_size

# 定义两个placeholder
x = tf.placeholder(tf.float32, [None,784])
y = tf.placeholder(tf.float32, [None,10])
keep_prob = tf.placeholder(tf.float32)

# 第一层神经元
# 用截断的正态分布来初始化,其标准差是0.1
W1 = tf.Variable(tf.truncated_normal([784, 2000],stddev=0.1))
# 偏置值初始化为0.1
b1 = tf.Variable(tf.zeros([2000])+0.1)
L1 = tf.nn.tanh(tf.matmul(x, W1)+b1)
# 调用封装好的dropout函数,其中的keep_prob参数就是设置让 百分之几的神经元工作,如：0.5就是50%的神经元工作
L1_drop = tf.nn.dropout(L1, keep_prob)

# 第二层神经元
W2 = tf.Variable(tf.truncated_normal([2000, 2000],stddev=0.1))
b2 = tf.Variable(tf.zeros([2000])+0.1)
L2 = tf.nn.tanh(tf.matmul(L1_drop, W2)+b2)
L2_drop = tf.nn.dropout(L2, keep_prob)

# 第三层神经元(用这么多层神经元,每层这么多神经元,就是为了模拟过拟合)
W3 = tf.Variable(tf.truncated_normal([2000, 1000],stddev=0.1))
b3 = tf.Variable(tf.zeros([1000])+0.1)
L3 = tf.nn.tanh(tf.matmul(L2_drop, W3)+b3)
L3_drop = tf.nn.dropout(L3, keep_prob)

# 第四层神经元(用这么多层神经元,每层这么多神经元,就是为了模拟过拟合)
W4 = tf.Variable(tf.truncated_normal([1000, 10],stddev=0.1))
b4 = tf.Variable(tf.zeros([10])+0.1)
prediction = tf.nn.softmax(tf.matmul(L3_drop, W4)+b4)


# 增加了隐藏层之后,参数变多了,更加难以收敛,如果想得到好的结果,必须要迭代很多次
# 隐藏层的神经元个数最好是2^n个

# 输入层与隐藏层之间的权重和偏置的初始化 隐藏层有5个神经元
# Weight_L1 = tf.Variable(tf.random_normal([784, 512]))
# biases_L1 = tf.Variable(tf.zeros([1, 512]))
# Wx_plus_b_L1 = tf.matmul(x, Weight_L1) + biases_L1
# L1 = tf.nn.tanh(Wx_plus_b_L1)

# 隐藏层和输出层之间权重和偏置的初始化
# Weight_L2 = tf.Variable(tf.random_normal([512, 10]))
# biases_L2 = tf.Variable(tf.zeros([1, 10]))
# Wx_plus_b_L2 = tf.matmul(L1, Weight_L2) + biases_L2
# prediction = tf.nn.softmax(Wx_plus_b_L2)

# 二次代价函数
# loss = tf.reduce_mean(tf.square(y-prediction))
# 现在改为对数似然函数来优化
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits=prediction))
# 梯度下降
train_step = tf.train.GradientDescentOptimizer(0.2).minimize(loss)

# 初始化变量
init = tf.global_variables_initializer()

# 这里求预测的正确数、结果存在一个布尔型的列表中
# equal(num1, num2)比较两个参数的大小是否一样,返回值为True或False
# arg_max() 求y(一维张量)这个结果最大的值的索引位置(如果标签值和预测值最大的值在同一个位置，则说明该条正确) 1表示按行查找
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(prediction, 1))

# 求准确率, cast()将布尔类型的列表中的元素转成32位的浮点类型,然后再求一个平均值(true=1.0 false=0)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

with tf.Session() as sess:
    sess.run(init)
    # 所有数据迭代21次
    for epoch in range(21):
        # 一次迭代中迭代计算好的数量个批次
        for batch in range(n_batch):
            # 获取100个批次保存在里面,数据保存在batch_xs中，标签保存在batchys中
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            # 将keep_prob 分别设置为1.0（所有神经元均工作） 和0.5等等进行比较
            sess.run(train_step, feed_dict={x:batch_xs, y:batch_ys, keep_prob:1.0})
        # 训练一个周期后,可以查看其准确率的变化
        # 查看训练集的准确率
        train_acc = sess.run(accuracy, feed_dict={x:mnist.train.images, y:mnist.train.labels, keep_prob:1.0})
        # 查看测试集的准确率
        test_acc = sess.run(accuracy, feed_dict={x:mnist.test.images, y:mnist.test.labels, keep_prob:1.0})
        print("第"+str(epoch+1)+"次迭代的训练集准确率: "+str(train_acc)+"; 测试集准确率: "+str(test_acc))

Extracting /Users/yuejinxiong/MNIST_data/train-images-idx3-ubyte.gz
Extracting /Users/yuejinxiong/MNIST_data/train-labels-idx1-ubyte.gz
Extracting /Users/yuejinxiong/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting /Users/yuejinxiong/MNIST_data/t10k-labels-idx1-ubyte.gz
第1次迭代的训练集准确率: 0.9576909; 测试集准确率: 0.9465
第2次迭代的训练集准确率: 0.9734; 测试集准确率: 0.9566
第3次迭代的训练集准确率: 0.9830545; 测试集准确率: 0.9625


KeyboardInterrupt: 